In [1]:
import cobra
import pandas as pd

model = cobra.io.read_sbml_model("2.2/finegoldia_magna_ATCC_29328_2.2.fo.ch.mp.mcb.lt.re.ar.gpr.pw.gf1.gfmm.gf2.gfco3.xml")

unwanted_metabolites = ["EX_o2_e"]

# import medium
snm3 = pd.read_csv("SNM3.csv", sep="\t")
snm3_dict = {f"EX_{met['BiGG']}_e" : 10.0 for i,met in snm3.iterrows()}
snm3_dict = {k: v for k,v in snm3_dict.items() if k not in unwanted_metabolites}
            
# For all exchanges open
for reac in model.exchanges:
    if reac.id in unwanted_metabolites:    # elimintes unwanted metabolites (O2)
        reac.lower_bound = 0.0
    else:
        reac.lower_bound = -1000.0

# Define SNM3 medium
#for reac in model.exchanges:
#    if reac.id in snm3_dict and reac.id not in unwanted_metabolites: 
#        reac.lower_bound = -10.0
#    else:
#        reac.lower_bound = 0.0

In [2]:
dissipation_rxns = pd.read_csv("energy_dissipation_rxns.csv") # see dissipation_reactions_from_Fritzemeier.xlsx
dissipation_rxns   

,type,equation
0,ATP,atp_c + h2o_c --> adp_c + h_c + pi_c
1,CTP,ctp_c + h2o_c --> cdp_c + h_c + pi_c
2,GTP,gtp_c + h2o_c --> gdp_c + h_c + pi_c
3,UTP,utp_c + h2o_c --> udp_c + h_c + pi_c
4,ITP,itp_c + h2o_c --> idp_c + h_c + pi_c
5,NADH,nadh_c --> h_c + nad_c
6,NADPH,nadph_c --> h_c + nadp_c
7,FADH2,fadh2_c --> 2 h_c + fad_c
8,FMNH2,fmnh2_c --> 2 h_c + fmn_c
9,Q8H2,q8h2_c --> 2 h_c + q8_c


In [3]:
import re
metabolites = []
for row in dissipation_rxns.iterrows():
    compounds = re.split(" &#8652; | ⇌ | <-> | --> | <-- ", row[1]['equation'])
    metabolites += re.split(" [+] ", compounds[0])
    metabolites += re.split(" [+] ", compounds[1])

met_id_list = [m.id for m in model.metabolites]
for m in metabolites:
    if " " in m:
        m = m.split(" ")[1]
    if m not in met_id_list:
        print(m)

2dmmq8_c


In [4]:
# The Metabolite 2dmmq8_c does not exist in model
dissipation_rxns.drop(11, inplace = True)
dissipation_rxns

,type,equation
0,ATP,atp_c + h2o_c --> adp_c + h_c + pi_c
1,CTP,ctp_c + h2o_c --> cdp_c + h_c + pi_c
2,GTP,gtp_c + h2o_c --> gdp_c + h_c + pi_c
3,UTP,utp_c + h2o_c --> udp_c + h_c + pi_c
4,ITP,itp_c + h2o_c --> idp_c + h_c + pi_c
5,NADH,nadh_c --> h_c + nad_c
6,NADPH,nadph_c --> h_c + nadp_c
7,FADH2,fadh2_c --> 2 h_c + fad_c
8,FMNH2,fmnh2_c --> 2 h_c + fmn_c
9,Q8H2,q8h2_c --> 2 h_c + q8_c


In [5]:
# from A. Grekova
# input : reaction string
# output : {<Metabolite object> : stoichiometry (integer)}
# this method is only useful, if coefficients are either 1 or 2 
def parse_reaction(eq):
    eq = eq.split(' ')
    eq_matrix={}
    
    are_products = False
    coeff = 1
    for i,part in enumerate(eq):
        if part == '-->':
            are_products = True
            continue          
        if part == '+':
            continue
        if part == '2':
            coeff = 2
            continue
        if are_products:
            eq_matrix[model.metabolites.get_by_id(part)] = 1*coeff
            coeff = 1
        else:
            eq_matrix[model.metabolites.get_by_id(part)] = -1*coeff
            coeff = 1
    return eq_matrix

In [6]:
from cobra import Reaction

#simulate changed conditions without changing the entire model
with model: 
    
    # add dissipation reactions
    for i, row in dissipation_rxns.iterrows():
        met_atp = parse_reaction(row['equation'])
        rxn = Reaction(row['type'])
        rxn.name = 'Test ' + row['type'] + ' dissipation reaction'
        rxn.add_metabolites(met_atp)
        model.add_reaction(rxn)
        
    for rxn in model.reactions:
        
        if 'EX_' in rxn.id:
            rxn.upper_bound = 0.0
            rxn.lower_bound = 0.0
            #print('Set exchange rxn to 0', rxn.name)
            
        # set reversible reactions fluxes to [-1,1]    
        elif rxn.reversibility:
            rxn.upper_bound = 1.0
            rxn.lower_bound = -1.0
            #print('Reversible rxn', rxn.name)
            
        # irreversible reactions have fluxes [1,0]    
        else:
            rxn.upper_bound = 1.0
            rxn.lower_bound = 0.0
            #print('Irreversible rxn', rxn.name)
     
    reactions_to_check = []
    # optimize by choosing one of dissipation reactions as an objective
    for i, row in dissipation_rxns.iterrows():
        model.objective = row['type']
        print('Set objective to', row['type'], ':', model.optimize().objective_value)
        
        if model.optimize().objective_value != 0.0:
            
            # check which reactions are active related to each metabolite that is produced after adding dissipation reactions
            fba_fluxes = pd.DataFrame(model.optimize().fluxes)
            fba_fluxes.reset_index(level=0,inplace=True)

            for react in list(fba_fluxes.loc[fba_fluxes['fluxes'] != 0]['index']):
                
                reactions_to_check.append(model.reactions.get_by_id(react).id)
                # print only active reactions
                print(model.reactions.get_by_id(react))
        
        print("----------------------------")

Set objective to ATP : 1.0
ADK1: amp_c + atp_c <=> 2.0 adp_c
DDCAt2pp: ddca_p + h_p --> ddca_c + h_c
PPK: atp_c + pi_c --> adp_c + ppi_c
FACOAE120: ddcacoa_c + h2o_c <=> coa_c + ddca_c + h_c
FACOAL120t2pp: atp_c + coa_c + ddca_p + h_p <=> amp_c + ddcacoa_c + h_c + ppi_c
ATP: atp_c + h2o_c --> adp_c + h_c + pi_c
----------------------------
Set objective to CTP : 1.0
DDCAt2pp: ddca_p + h_p --> ddca_c + h_c
PPDK: atp_c + pi_c + pyr_c --> amp_c + h_c + pep_c + ppi_c
PYK4: cdp_c + h_c + pep_c --> ctp_c + pyr_c
FACOAE120: ddcacoa_c + h2o_c <=> coa_c + ddca_c + h_c
FACOAL120t2pp: atp_c + coa_c + ddca_p + h_p <=> amp_c + ddcacoa_c + h_c + ppi_c
CTP: ctp_c + h2o_c --> cdp_c + h_c + pi_c
----------------------------
Set objective to GTP : 1.0
ADK1: amp_c + atp_c <=> 2.0 adp_c
DDCAt2pp: ddca_p + h_p --> ddca_c + h_c
PPK: atp_c + pi_c --> adp_c + ppi_c
PPK50_B: 47.0 gdp_c + ppi50_c --> 47.0 gtp_c + pppi_c
PPK50r: 47.0 atp_c + pppi_c <=> 47.0 adp_c + ppi50_c
FACOAE120: ddcacoa_c + h2o_c <=> coa_c 

In [7]:
# ADK1: is reversible
# DDCAt2pp: not in MetaCyc
# FACOAE120: ddcacoa_c + h2o_c <=> coa_c + ddca_c + h_c  =====> ddcacoa_c + h2o_c --> coa_c + ddca_c + h_c
model.reactions.get_by_id("FACOAE120").build_reaction_from_string("ddcacoa_c + h2o_c --> coa_c + ddca_c + h_c")
model.reactions.get_by_id("FACOAE120")
# FACOAL120t2pp: atp_c + coa_c + ddca_p + h_p <=> amp_c + ddcacoa_c + h_c + ppi_c ====> atp_c + coa_c + ddca_p + h_p --> amp_c + ddcacoa_c + h_c + ppi_c
model.reactions.get_by_id("FACOAL120t2pp").build_reaction_from_string("atp_c + coa_c + ddca_p + h_p --> amp_c + ddcacoa_c + h_c + ppi_c")
model.reactions.get_by_id("FACOAL120t2pp")
# ACKr: is reversible (Acetate kinase)
# PPK50r: is reversible
# PTAr: is reversible

Reaction identifier,FACOAL120t2pp
Name,Fatty-acid-CoA ligase (dodecanoate transport via vectoral Co-A coupling)
Memory address,0x07fe139f2af40
Stoichiometry,atp_c + coa_c + ddca_p + h_p --> amp_c + ddcacoa_c + h_c + ppi_c ATP C10H12N5O13P3 + Coenzyme A + Dodecanoate (n-C12:0) + H+ --> AMP C10H12N5O7P + Dodecanoyl-CoA (n-C12:0CoA) + H+ + Diphosphate
GPR,FMG_1293
Lower bound,0
Upper bound,1000.0


In [8]:
model.slim_optimize()

94.24459770349941

In [9]:
from cobra import Reaction

#simulate changed conditions without changing the entire model
with model: 
    
    # add dissipation reactions
    for i, row in dissipation_rxns.iterrows():
        met_atp = parse_reaction(row['equation'])
        rxn = Reaction(row['type'])
        rxn.name = 'Test ' + row['type'] + ' dissipation reaction'
        rxn.add_metabolites(met_atp)
        model.add_reaction(rxn)
        
    for rxn in model.reactions:
        
        if 'EX_' in rxn.id:
            rxn.upper_bound = 0.0
            rxn.lower_bound = 0.0
            #print('Set exchange rxn to 0', rxn.name)
            
        # set reversible reactions fluxes to [-1,1]    
        elif rxn.reversibility:
            rxn.upper_bound = 1.0
            rxn.lower_bound = -1.0
            #print('Reversible rxn', rxn.name)
            
        # irreversible reactions have fluxes [1,0]    
        else:
            rxn.upper_bound = 1.0
            rxn.lower_bound = 0.0
            #print('Irreversible rxn', rxn.name)
     
    reactions_to_check = []
    # optimize by choosing one of dissipation reactions as an objective
    for i, row in dissipation_rxns.iterrows():
        model.objective = row['type']
        print('Set objective to', row['type'], ':', model.optimize().objective_value)
        
        if model.optimize().objective_value != 0.0:
            
            # check which reactions are active related to each metabolite that is produced after adding dissipation reactions
            fba_fluxes = pd.DataFrame(model.optimize().fluxes)
            fba_fluxes.reset_index(level=0,inplace=True)

            for react in list(fba_fluxes.loc[fba_fluxes['fluxes'] != 0]['index']):
                
                reactions_to_check.append(model.reactions.get_by_id(react).id)
                # print only active reactions
                print(model.reactions.get_by_id(react))
        
        print("----------------------------")

Set objective to ATP : 0.0
----------------------------
Set objective to CTP : 0.0
----------------------------
Set objective to GTP : 0.0
----------------------------
Set objective to UTP : 0.0
----------------------------
Set objective to ITP : 0.0
----------------------------
Set objective to NADH : 0.0
----------------------------
Set objective to NADPH : 0.0
----------------------------
Set objective to FADH2 : 0.0
----------------------------
Set objective to FMNH2 : 0.0
----------------------------
Set objective to Q8H2 : 0.0
----------------------------
Set objective to MQL8 : 0.0
----------------------------
Set objective to ACCOA : 0.0
----------------------------
Set objective to GLU : 0.0
----------------------------
Set objective to PROTON : 0.0
----------------------------


#### => No more circles

In [10]:
cobra.io.write_sbml_model(model, "2.2/finegoldia_magna_ATCC_29328_2.2.fo.ch.mp.mcb.lt.re.ar.gpr.pw.gf1.gfmm.gf2.gfco3.circ.xml")